In [18]:
# import tensorflow as tf
import tensorflow as tf

In [ ]:
# Lokasi folder dataset yang sesuai
train_dir = "archive (9)/hymenoptera_data/train"
val_dir = "archive (9)/hymenoptera_data/val"

# Load dataset dari folder (pakai tf.keras.utils)
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),
    batch_size=32
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    image_size=(224, 224),
    batch_size=32
)


AttributeError: module 'tensorflow' has no attribute 'keras'